# Semantic Kernel နှင့် OpenBnB MCP Server ပေါင်းစည်းမှု

ဒီ notebook က Semantic Kernel ကို OpenBnB MCP server နဲ့ပေါင်းစပ်ပြီး MCPStdioPlugin ကိုသုံးကာ အမှန်တကယ်ရှိတဲ့ Airbnb အိမ်ခန်းတွေကို ရှာဖွေဖို့ ပြသပေးမှာ ဖြစ်ပါတယ်။ LLM Access အတွက် Azure AI Foundry ကို အသုံးပြုထားပါတယ်။ သင့်ပတ်ဝန်းကျင် variable တွေကို စီစဉ်ဖို့အတွက် [Setup Lesson](/00-course-setup/README.md) ကို လိုက်နာနိုင်ပါတယ်။


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP Plugin ချိတ်ဆက်မှု ဖန်တီးခြင်း

ကျွန်တော်တို့ [OpenBnB MCP server](https://github.com/openbnb-org/mcp-server-airbnb) ကို MCPStdioPlugin အသုံးပြု၍ ချိတ်ဆက်ပါမည်။ ဒီ server သည် @openbnb/mcp-server-airbnb package ကို အသုံးပြု၍ Airbnb ရှာဖွေမှု လုပ်ဆောင်နိုင်စွမ်းပေးပါသည်။


## Client ကိုဖန်တီးခြင်း

ဤနမူနာတွင် LLM ကို အသုံးပြုရန် Azure AI Foundry ကို အသုံးပြုမည်ဖြစ်သည်။ သင့်ပတ်ဝန်းကျင်အပြောင်းအလဲများကို မှန်ကန်စွာ စီစဉ်ထားရှိပါ။


## ပတ်ဝန်းကျင် ပြင်ဆင်မှု

Azure OpenAI ဆက်တင်များကို ပြင်ဆင်ပါ။ အောက်ပါ ပတ်ဝန်းကျင် အပြောင်းအလဲများကို သတ်မှတ်ထားရှိရမည်ဖြစ်သည်။
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP အစိတ်အပိုင်း ပေါင်းစည်းမှုကို နားလည်ခြင်း

ဒီ notebook က **အမှန်တကယ် OpenBnB MCP server** ကို ချိတ်ဆက်ပြီး Airbnb ရှာဖွေမှုလုပ်ဆောင်ချက်ကို ပေးစွမ်းပါတယ်။

### ဘယ်လိုအလုပ်လုပ်သလဲ:

1. **MCPStdioPlugin**: MCP server နဲ့ standard input/output အဆက်အသွယ်ကို အသုံးပြုပါတယ်
2. **Real NPM Package**: `@openbnb/mcp-server-airbnb` ကို npx နဲ့ download လုပ်ပြီး run လုပ်ပါတယ်
3. **Live Data**: Airbnb API တွေကနေ အမှန်တကယ် property data တွေကို ပြန်ပေးပါတယ်
4. **Function Discovery**: MCP server က ရရှိနိုင်တဲ့ function တွေကို agent က အလိုအလျောက် ရှာဖွေပါတယ်

### ရရှိနိုင်တဲ့ Function တွေ:

OpenBnB MCP server က အောက်ပါ function တွေကို ပုံမှန်ပေးပါတယ်:
- **search_listings** - တည်နေရာနဲ့ criteria အပေါ်မူတည်ပြီး Airbnb property တွေကို ရှာဖွေပါ
- **get_listing_details** - အထူး property တွေကို အသေးစိတ်အချက်အလက်ရယူပါ
- **check_availability** - သတ်မှတ်ရက်များအတွက် ရရှိနိုင်မှုကို စစ်ဆေးပါ
- **get_reviews** - Property တွေအတွက် ပြန်လည်သုံးသပ်ချက်များကို ရယူပါ
- **get_host_info** - Property host များအကြောင်းအချက်အလက်ကို ရယူပါ

### လိုအပ်ချက်များ:

- **Node.js** ကို သင့်စနစ်မှာ install လုပ်ထားရမယ်
- MCP server package ကို download လုပ်ဖို့ **Internet connection** လိုအပ်ပါတယ်
- **NPX** ရရှိနိုင်ဖို့ (Node.js နဲ့အတူပါဝင်ပါတယ်)

### အဆက်အသွယ်ကို စမ်းသပ်ခြင်း:

MCP server ကို လက်တွေ့စမ်းသပ်ဖို့ အောက်ပါ command ကို run လုပ်နိုင်ပါတယ်:
```bash
npx -y @openbnb/mcp-server-airbnb
```

ဒီ command က OpenBnB MCP server ကို download လုပ်ပြီး start လုပ်ပါမယ်၊ အဲဒီနောက် Semantic Kernel က အမှန်တကယ် Airbnb data ကို ချိတ်ဆက်ပါမယ်။


## OpenBnB MCP Server နှင့် Agent ကို အလုပ်လုပ်စေခြင်း

အခုတော့ OpenBnB MCP server ကို ချိတ်ဆက်ပြီး AI Agent ကို အလုပ်လုပ်စေပါမယ်။ ဒီ Agent က Stockholm မှာ လူကြီး ၂ ယောက်နဲ့ ကလေး ၁ ယောက်အတွက် အစစ်အမှန် Airbnb အိမ်ခြံမြေတွေကို ရှာဖွေမှာ ဖြစ်ပါတယ်။ `user_inputs` စာရင်းကို ပြောင်းလဲပြီး ရှာဖွေမှုအချက်အလက်တွေကို ပြင်ဆင်နိုင်ပါတယ်။


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

အကျဉ်းချုပ်  
ဂုဏ်ပြုပါတယ်! သင်သည် Model Context Protocol (MCP) ကို အသုံးပြု၍ အိမ်ရာရှာဖွေရေး၏ အပြင်လောကဆိုင်ရာလုပ်ငန်းစဉ်နှင့် ပေါင်းစည်းထားသော AI အေးဂျင့်တစ်ခုကို အောင်မြင်စွာ တည်ဆောက်နိုင်ခဲ့ပါပြီ။

အသုံးပြုထားသော နည်းပညာများ:  
- Semantic Kernel - Azure OpenAI ဖြင့် ထူးချွန်သော အေးဂျင့်များ တည်ဆောက်ရန်  
- Azure AI Foundry - LLM စွမ်းရည်များနှင့် စကားဝိုင်းဖြည့်စွက်မှုအတွက်  
- MCP (Model Context Protocol) - စံပြပုံစံဖြင့် ကိရိယာများ ပေါင်းစည်းရန်  
- OpenBnB MCP Server - အမှန်တကယ် Airbnb ရှာဖွေရေး လုပ်ဆောင်နိုင်စွမ်းအတွက်  
- Node.js/NPX - ပြင်ပ MCP server ကို အလုပ်လုပ်စေရန်  

သင်လေ့လာခဲ့သည့်အရာများ:  
- MCP ပေါင်းစည်းမှု: Semantic Kernel အေးဂျင့်များကို ပြင်ပ MCP server များနှင့် ချိတ်ဆက်ခြင်း  
- အချိန်နှင့်တပြေးညီ ဒေတာရယူမှု: API များမှ အမှန်တကယ် Airbnb အိမ်ရာများ ရှာဖွေရန်  
- ပရိုတိုကော ဆက်သွယ်မှု: အေးဂျင့်နှင့် MCP server အကြား stdio ဆက်သွယ်မှု အသုံးပြုခြင်း  
- လုပ်ဆောင်ချက် ရှာဖွေမှု: MCP server များမှ ရရှိနိုင်သော လုပ်ဆောင်ချက်များကို အလိုအလျောက် ရှာဖွေခြင်း  
- တိုက်ရိုက် တုံ့ပြန်မှုများ: လုပ်ဆောင်ချက်ခေါ်ဆိုမှုများကို အချိန်နှင့်တပြေးညီ ဖမ်းယူပြီး မှတ်တမ်းတင်ခြင်း  
- HTML Rendering: အေးဂျင့်၏ တုံ့ပြန်မှုများကို စတိုင်ပြုထားသော ဇယားများနှင့် အပြန်အလှန် ပြသမှုများဖြင့် ဖော်ပြခြင်း  

နောက်တစ်ဆင့်များ:  
- ထပ်မံသော MCP server များ (ရာသီဥတု, လေယာဉ်, စားသောက်ဆိုင်) ပေါင်းစည်းခြင်း  
- MCP နှင့် A2A ပရိုတိုကောများ ပေါင်းစည်းထားသော မျိုးစုံအေးဂျင့်စနစ် တည်ဆောက်ခြင်း  
- သင့်ကိုယ်ပိုင် ဒေတာရင်းမြစ်များအတွက် အထူးပြု MCP server များ ဖန်တီးခြင်း  
- အစည်းအဝေးများအကြား ဆက်လက်စကားဝိုင်းမှတ်ဉာဏ် ထည့်သွင်းခြင်း  
- MCP server စီမံခန့်ခွဲမှုဖြင့် အေးဂျင့်ကို Azure Functions တွင် တင်ပို့ခြင်း  
- အသုံးပြုသူ အတည်ပြုမှုနှင့် ကြိုတင်မှာယူနိုင်စွမ်း ထည့်သွင်းခြင်း  

MCP Architecture ၏ အဓိကအားသာချက်များ:  
- စံပြပုံစံ: AI အေးဂျင့်များကို ပြင်ပကိရိယာများနှင့် ချိတ်ဆက်ရန် စံပြပရိုတိုကော  
- အချိန်နှင့်တပြေးညီ ဒေတာ: ဝန်ဆောင်မှုအမျိုးမျိုးမှ အသစ်ဆုံး၊ မှန်ကန်သော အချက်အလက်များ ရယူနိုင်ခြင်း  
- တိုးချဲ့နိုင်စွမ်း: ဒေတာရင်းမြစ်အသစ်များနှင့် ကိရိယာများကို လွယ်ကူစွာ ပေါင်းစည်းနိုင်ခြင်း  
- ပေါင်းစည်းနိုင်စွမ်း: AI ဖရိမ်းဝက်များနှင့် အေးဂျင့်ပလက်ဖောင်းအမျိုးမျိုးအကြား လုပ်ဆောင်နိုင်ခြင်း  
- တာဝန်ခွဲခြားမှု: AI လိုဂစ်နှင့် ပြင်ပဒေတာရယူမှုအကြား ရှင်းလင်းသော ခွဲခြားမှု  



---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်ခြင်းတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါရှိနိုင်သည်ကို သတိပြုပါ။ မူရင်းဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာရှိသော ရင်းမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များကို အသုံးပြုရန် အကြံပြုပါသည်။ ဤဘာသာပြန်ကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအလွတ်များ သို့မဟုတ် အနားလွဲမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
